In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import copy
import tqdm
import IProgress
from hfunc import models
from hfunc import metrics

In [3]:
fashion_mnist = tf.keras.datasets.fashion_mnist
class_accuracy = metrics.ClassAccuracy()

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10)
loss, acc = model.evaluate(x_test, y_test, verbose=2)
end_not_reached = True
improved = False
size = 128
tol = -1e-30
current_pos = 0
best_pos = -1
best_change = tol
original = model.get_weights()
bas = [acc]
bls = [loss]
best_weights = model.get_weights()
nodes_removed = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed = 0
while end_not_reached or improved:
    if not(end_not_reached):
        end_not_reached = True
        improved = False
        current_pos = 0
        size -= 1
        nodes_removed += [best_pos]
        best_weights[0][:,best_pos] = 0
        best_weights[1][best_pos] = 0
        best_weights[2][best_pos,:] = 0
        best_pos = -1
        tol -= best_change
        ol = best_loss
        oa = best_acc
        bas += [best_acc]
        bls += [best_loss]
        print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
        best_change = tol
        num_removed += 1
    if current_pos in nodes_removed:
        current_pos += 1
        if current_pos - num_removed >= size:
            end_not_reached = False
        continue
    w = copy.deepcopy(best_weights)
    w[0][:,current_pos] = 0
    w[1][current_pos] = 0
    w[2][current_pos,:] = 0
    model.set_weights(w)
    nl, na = model.evaluate(x_test, y_test, verbose=0)
    if 0.8*(na - oa) + 0.2*(ol - nl) >= best_change:
        best_change = 0.8*(na - oa) + 0.2*(ol - nl)
        print(best_change)
        best_pos = current_pos
        improved = True
        best_acc = na
        best_loss = nl
        print("Found something better")
    current_pos += 1
    if current_pos - num_removed >= size:
        end_not_reached = False
    if current_pos%20 == 0:
        print("Did 20 iterations")

model.set_weights(best_weights)
loss2, acc2 = model.evaluate(x_test, y_test, verbose=2)

In [ ]:
print(loss - loss2)
print(acc2 - acc)
print((loss - loss2)/loss * 100)
print((acc2 - acc)/acc * 100)
print(num_removed)
print(best_weights[1])

In [ ]:
or_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
or_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
or_model.set_weights(original)
print(or_model.get_weights()[1])

In [ ]:
model.evaluate(x_train, y_train, verbose=2)

In [ ]:
or_model.evaluate(x_train, y_train, verbose=2)

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

In [ ]:
or_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
red_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128-num_removed, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
red_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
red_model.fit(x_train, y_train, epochs=100, verbose=1)
red_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
red_model.fit(x_train, y_train, epochs=15, verbose=1)
red_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
import os
from tensorflow import keras
import datetime

In [ ]:
reduced_path = f'C:/Users/lucas/Documents/Masters/models/reduced/fashion_mnist_128_{128-num_removed}'
original_path = f'C:/Users/lucas/Documents/Masters/models/orininal/fashion_mnist_128_{128-num_removed}'
full_reduced_path = f'C:/Users/lucas/Documents/Masters/models/full_reduced/fashion_mnist_128_{128-num_removed}'

In [ ]:
model.save_weights(reduced_path)

In [ ]:
or_model.save_weights(original_path)

In [ ]:
new_weights = [np.zeros((best_weights[0].shape[0], best_weights[0].shape[1] - num_removed)), np.zeros((best_weights[1].shape[0] - num_removed)), np.zeros((best_weights[2].shape[0] - num_removed, best_weights[2].shape[1])), best_weights[3]]

j = 0
for i in range(len(best_weights[1])):
    if i not in nodes_removed:
        new_weights[0][:, j] = best_weights[0][:, i]
        new_weights[1][j] = best_weights[1][i]
        new_weights[2][j, :] = best_weights[2][i, :]
        j = j + 1

In [ ]:
red_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(12, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    
red_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
red_model.set_weights(new_weights)
red_model.save_weights(full_reduced_path)
red_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
red_model.set_weights(new_weights)
red_model.fit(x_train, y_train, epochs=15, verbose=1, batch_size=4096)
red_model.evaluate(x_test, y_test, verbose=2, batch_size=256)

In [ ]:
new_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_model.set_weights(best_weights)
new_model.evaluate(x_test, y_test, verbose=2)
new_model.fit(x_train, y_train, epochs=10, verbose=1, batch_size=2048)
new_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
loss, acc = new_model.evaluate(x_test, y_test, verbose=2)
end_not_reached = True
improved = False
size = 128
tol = -1e-30
current_pos = 0
best_pos = -1
best_change = tol
original2 = new_model.get_weights()
bas2 = [acc]
bls2 = [loss]
best_weights2 = new_model.get_weights()
nodes_removed2 = []
best_acc = 0
best_loss = 1e20
ol = loss
oa = acc
num_removed2 = 0
while end_not_reached or improved:
    if not(end_not_reached):
        end_not_reached = True
        improved = False
        current_pos = 0
        size -= 1
        nodes_removed2 += [best_pos]
        best_weights2[0][:,best_pos] = 0
        best_weights2[1][best_pos] = 0
        best_weights2[2][best_pos,:] = 0
        best_pos = -1
        tol -= best_change
        ol = best_loss
        oa = best_acc
        bas2 += [best_acc]
        bls2 += [best_loss]
        print("Improvement has occured!! Accuracy:", best_acc, "--- Loss:", best_loss, '--- Change:', best_change, '--- New tol:', tol)
        best_change = tol
        num_removed2 += 1
    if current_pos in nodes_removed2:
        current_pos += 1
        if current_pos - num_removed2 >= size:
            end_not_reached = False
        continue
    w = copy.deepcopy(best_weights2)
    w[0][:,current_pos] = 0
    w[1][current_pos] = 0
    w[2][current_pos,:] = 0
    new_model.set_weights(w)
    nl, na = new_model.evaluate(x_test, y_test, verbose=0)
    print(f"Node {current_pos}:", 0.8*(na - oa) + 0.2*(ol - nl))
    if 0.8*(na - oa) + 0.2*(ol - nl) > best_change:
        best_change = 0.8*(na - oa) + 0.2*(ol - nl)
        print(best_change)
        best_pos = current_pos
        improved = True
        best_acc = na
        best_loss = nl
        print("Found something better")
    current_pos += 1
    if current_pos - num_removed2 >= size:
        end_not_reached = False
    if current_pos%20 == 0:
        print("Did 20 iterations")

new_model.set_weights(best_weights2)
loss2, acc2 = new_model.evaluate(x_test, y_test, verbose=2)

In [ ]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
l, a = or_model.evaluate(x_test, y_test, verbose=2)
or_weights = or_model.get_weights()
size = 128
for i in range(128):
    w = copy.deepcopy(or_weights)
    w[0][:,i] = 0
    w[1][i] = 0
    w[2][i,:] = 0
    tester_model.set_weights(w)
    nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
    print(f"Node {i}:", 0.6*(na - a) + 0.4*(l - nl))

In [ ]:
num_test = 20
num_zeros = np.zeros(num_test)
num_worse = np.zeros(num_test)
num_important = np.zeros(num_test)
losses = np.zeros(num_test)
accs = np.zeros(num_test)
zero_nodes = []
worsening_nodes = []
important_nodes = []
tol = -1e-4
for j in range(num_test):
    blank_model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
    blank_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    blank_model.fit(x_train, y_train, epochs=10)
    l, a = blank_model.evaluate(x_test, y_test, verbose=2)
    losses[j] = l
    accs[j] = a
    z = []
    wr = []
    imp = []
    for i in range(128):
        w = blank_model.get_weights()
        w[0][:,i] = 0
        w[1][i] = 0
        w[2][i,:] = 0
        tester_model.set_weights(w)
        nl, na = tester_model.evaluate(x_test, y_test, verbose=0)
        change = 0.8*(na - a) + 0.2*(l - nl)
        if change <= 0 and change >= tol:
            num_zeros[j] += 1
            z += [i]
        elif change > 0:
            num_worse[j] += 1
            wr += [i]
        else:
            num_important[j] += 1
            imp += [i]
    zero_nodes += [z]
    worsening_nodes += [wr]
    important_nodes += [imp]

In [ ]:
print(num_zeros)
print(num_worse)
print(num_important)